In [1]:
import pandas as pd

In [2]:
pd.__version__

'2.3.3'

In [3]:
DATE_LIMIT = pd.Timestamp('2020-01-01')
ALL_PREP_PET_NAME = 'All_Preprocessed_PET.csv'
DXSUM_NAME = 'All_Subjects_DXSUM.csv'
CDR_NAME = 'All_Subjects_CDR.csv'

In [4]:
def create_merged_table(all_prep_pet_name, dxsum_name, cdr_name):
    '''
    Merge the PET metadata table, DXSUM (AD diagnosis info) table, and CDR score table. Crucial for selecting only images with
    an associated CDR score and diagnosis.
    '''
    all_prep_pet = pd.read_csv(all_prep_pet_name, parse_dates=['image_date'])
    dxsum = pd.read_csv(dxsum_name, parse_dates=['EXAMDATE'])
    cdr = pd.read_csv(cdr_name, parse_dates=['VISDATE'])
    
    # Merge diagnosis table with PET info table by checking for closest diagnosis exam date to each image date
    def get_closest_dx_date(row):
        deltas = (dxsum[dxsum['PTID'] == row['subject_id']]['EXAMDATE'] - row['image_date']).abs()
        return deltas.idxmin() if not deltas.isna().all() else pd.NA

    deltas = all_prep_pet.apply(get_closest_dx_date, axis=1)
    data_init = all_prep_pet.copy()
    data_init['closest_dx_ind'] = deltas
    data_init = pd.merge(data_init, dxsum, left_on='closest_dx_ind', right_index=True).dropna(subset=['closest_dx_ind'])

    # Merge again with CDR table by searching for CDR visit date closest to the image date
    def get_closest_cdr_date(row):
        deltas = (cdr[cdr['PTID'] == row['PTID']]['VISDATE'] - row['image_date']).abs()
        return deltas.idxmin() if not deltas.isna().all() else pd.NA

    deltas = data_init.apply(get_closest_cdr_date, axis=1)
    data_init['closest_cdr_ind'] = deltas
    data = pd.merge(data_init, cdr, left_on='closest_cdr_ind', right_index=True).dropna(subset=['closest_cdr_ind', 'CDRSB']) 
    return data

In [11]:
data = create_merged_table(ALL_PREP_PET_NAME, DXSUM_NAME, CDR_NAME)

In [13]:
# ad = data[(data['DIAGNOSIS'] == 3.0) & (data['image_date'] < DATE_LIMIT)]
# (ad['image_date'] - ad['EXAMDATE']).abs().sort_values()

In [14]:
# data[data['image_date'] < pd.Timestamp('2020-01-01')]['subject_id'].unique().shape

In [15]:
# data[data['image_date'] < pd.Timestamp('2020-01-01')]['DIAGNOSIS'].value_counts()
# data['DIAGNOSIS'].value_counts()

In [16]:
def write_img_ids(data, date_limit):
    '''
    Write the image IDs to a text file separated by commas. Makes it easier to copy-paste the IDs into ADNI ARC builder 
    to download the images.
    '''
    cohorts = data[data['image_date'] < date_limit]

    cn_cohort = cohorts[cohorts['DIAGNOSIS'] == 1.0]['image_id']
    mci_cohort = cohorts[cohorts['DIAGNOSIS'] == 2.0]['image_id']
    ad_cohort = cohorts[cohorts['DIAGNOSIS'] == 3.0]['image_id']
    
    print(len(cn_cohort))
    print(len(mci_cohort))
    print(len(ad_cohort))
    return

    with open('cn_cohort.txt', 'w') as f:
        f.write(','.join(cn_cohort.astype(str)))
        
    with open('mci_cohort.txt', 'w') as f:
        f.write(','.join(mci_cohort.astype(str)))
        
    with open('ad_cohort.txt', 'w') as f:
        f.write(','.join(ad_cohort.astype(str)))
        
def write_all_cohorts(data, date_limit):
    '''
    Write the fully merged data table to a CSV file. Limit by a maximum date
    '''
    all_cohorts_fname = '/home/hice1/khom9/CS8903/all_cohorts_cvae.csv'
    data[data['image_date'] < date_limit].to_csv(all_cohorts_fname)

In [17]:
# Writes image IDs to a txt file to copy into ADNI ARC builder
write_img_ids(data, DATE_LIMIT)

1036
1781
842


In [9]:
# Write fully joined table containing cohort data
# write_all_cohorts(data, DATE_LIMIT)

In [10]:
# data[data['image_id'] == 1288913]

In [11]:
# data['CDRSB'].isna().sum()

In [13]:
data[data['image_date'] < DATE_LIMIT]['DIAGNOSIS'].value_counts()

2.0    1781
1.0    1036
3.0     842
Name: DIAGNOSIS, dtype: int64